# Points of interest based bicycle network generation
## Project: Bicycle network analysis with Gourab, Sayat, Tyler, Michael, Roberta

This notebook follows the transit-oriented development approach of palominos2020ica and applies cardillo2006spp: Take the greedy triangulation between railway/underground stations (or other points of interest created in 02_prepare_pois). This is the cold start bicycle network generation process which creates bicycle networks from scratch.

Contact: Michael Szell (michael.szell@gmail.com)  
Created: 2020-06-18  
Last modified: 2020-07-14

## Preliminaries

### Parameters

In [ ]:
debug = True # If True, will produce plots and/or verbose output to double-check

poi_source = "railwaystation" # railwaystation, grid, popdensity
prune_measure = "betweenness"

### Setup

In [ ]:
%run -i path.py
%run -i setup.py

%matplotlib inline
%load_ext watermark
%watermark -n -v -m -g -iv

### Functions

In [ ]:
%run -i functions.py

## Plot the starting point

In [ ]:
# if debug:
#     my_plot_reset(G_carall, nnids)
#     ig.plot(G_carall, PATH["plots"] + placeid + '_carall_poi_' + poi_source + ".png", bbox=(800,800))
#     ig.plot(G_carall)

## Routing (shortest paths)

In [ ]:
for placeid, placeinfo in cities.items():
    print(placeid + ": Generating networks")
    placepath = PATH["data"] + placeid + "/"

    # Load networks
    G_carall = csv_to_ig(placepath, placeid, 'carall')
    
    # Load POIs
    with open(placepath + placeid + '_poi_' + poi_source + '_nnidscarall.csv') as f:
        nnids = [int(line.rstrip()) for line in f]
    
    # Generation
    (GTs, GT_abstracts) = greedy_triangulation_routing(G_carall, nnids, prune_quantiles, prune_measure)
    
    # Write results
    results = {"placeid": placeid, "prune_measure": prune_measure, "poi_source": poi_source, "prune_quantiles": prune_quantiles, "GTs": GTs, "GT_abstracts": GT_abstracts}

    filename = placeid + '_poi_' + poi_source + "_" + prune_measure
    resultfile = open(PATH["results"] + filename + ".pickle", 'wb')
    pickle.dump(results, resultfile)
    resultfile.close()

## Plot results

### Plot the abstract Greedy Triangulation

In [ ]:
if debug: 
    for GT_abstract, prune_quantile in zip(GT_abstracts, prune_quantiles):
        color_nodes(GT_abstract, "red", False, False)
        size_nodes(GT_abstract, 4, False, False)
        ig.plot(GT_abstract, PATH["plots"] + placeid + '_GTabstract_poi_'+ poi_source + "_" + prune_measures[prune_measure] + "{:.2f}".format(prune_quantile) + '.png', bbox=(800,800))

### Plot just the bicycle network

In [ ]:
if debug:
    for GT, prune_quantile in zip(GTs, prune_quantiles):
        size_nodes(GT, 0)
        width_edges(GT, 2)
        color_edges(GT, "blue")
        ig.plot(GT, PATH["plots"] + placeid + '_GTbonly_poi_' + poi_source + "_" + prune_measures[prune_measure] + "{:.2f}".format(prune_quantile) + '.png', bbox=(800,800))

### Plot all together

In [ ]:
if debug:
    for GT, prune_quantile in zip(GTs, prune_quantiles):
        my_plot_reset(G_carall, nnids)
        color_nodes(G_carall, "red", nnids)
        size_nodes(G_carall, 8, nnids)
        GT_ids = [v["id"] for v in GT.vs]  
        color_nodes(G_carall, "blue", GT_ids)
        size_nodes(G_carall, 5, GT_ids)
        ig.plot(G_carall, PATH["plots"] + placeid + '_GTall_poi_' + poi_source + "_" + prune_measures[prune_measure] + "{:.2f}".format(prune_quantile) + '.png', bbox=(800,800))